In [2]:
import csv

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [3]:
# Specify data paths
dataset = 'new_gestures.csv'
model_save_path = 'keypoint_classifier.hdf5'
tflite_save_path = 'keypoint_classifier.tflite'

# Set number of classes

In [4]:
# Change training classes if necessary
NUM_CLASSES = 12

# Dataset reading

In [5]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [6]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [8]:
# Classes count
counts = np.unique(y_dataset, return_counts=True)
df = pd.DataFrame(counts)
df.T.plot(kind="bar", stacked=True)
print(counts)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32), array([1595, 1661,  100,  672,  431,  315,  139,  190,  421, 1356,  257,
        302]))


# Model building

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.0),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.0),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.0),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [10]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 32)                1376      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 12)                2

In [11]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False, save_best_only=True)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=50, verbose=1)

In [12]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [13]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
76/88 [========================>.....] - ETA: 0s - loss: 2.1588 - accuracy: 0.2829
Epoch 1: val_loss improved from inf to 1.73526, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 3s 9ms/step - loss: 2.1109 - accuracy: 0.3004 - val_loss: 1.7353 - val_accuracy: 0.4699
Epoch 2/1000
42/88 [=============>................] - ETA: 0s - loss: 1.5641 - accuracy: 0.4933

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


65/88 [=====================>........] - ETA: 0s - loss: 1.4925 - accuracy: 0.5010
Epoch 2: val_loss improved from 1.73526 to 1.12020, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 3ms/step - loss: 1.4166 - accuracy: 0.5114 - val_loss: 1.1202 - val_accuracy: 0.5973
Epoch 3/1000
86/88 [============================>.] - ETA: 0s - loss: 0.9308 - accuracy: 0.6873
Epoch 3: val_loss improved from 1.12020 to 0.74051, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 4ms/step - loss: 0.9316 - accuracy: 0.6879 - val_loss: 0.7405 - val_accuracy: 0.8000
Epoch 4/1000
86/88 [============================>.] - ETA: 0s - loss: 0.6204 - accuracy: 0.8528
Epoch 4: val_loss improved from 0.74051 to 0.50222, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 4ms/step - loss: 0.6176 - accuracy: 0.8539 - val_loss: 0.5022 - val_accuracy: 0.8763
Epoch 5/1000
79/88 [=========================>....] - ETA: 

In [14]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [15]:
# Model evaluation
# TODO Test on loaded model
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=64)

30/30 [==============================] - 0s 2ms/step - loss: 0.0446 - accuracy: 0.9930


In [16]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 124ms/step
[2.11532298e-03 4.75956909e-08 4.90263392e-06 1.07304644e-04
 1.43886325e-07 9.97734308e-01 7.38590363e-17 2.02929051e-09
 8.38584790e-13 3.70644339e-05 3.55336754e-16 9.80118216e-07]
5


# Confusion matrix

In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

59/59 [==============================] - 0s 1ms/step


Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       427
           1       0.99      1.00      0.99       412
           2       1.00      1.00      1.00        23
           3       1.00      1.00      1.00       149
           4       1.00      0.97      0.99       116
           5       1.00      1.00      1.00        79
           6       1.00      1.00      1.00        42
           7       1.00      1.00      1.00        43
           8       1.00      0.98      0.99       110
           9       0.99      0.99      0.99       323
          10       0.95      1.00      0.98        63
          11       0.99      0.99      0.99        73

    accuracy                           0.99      1860
   macro avg       0.99      0.99      0.99      1860
weighted avg       0.99      0.99      0.99      1860



# Convert to model for Tensorflow-Lite

In [18]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/kf/9vc6x6n97q942grwf1zf0trw0000gn/T/tmpg96ejf4t/assets


INFO:tensorflow:Assets written to: /var/folders/kf/9vc6x6n97q942grwf1zf0trw0000gn/T/tmpg96ejf4t/assets
2023-11-30 21:49:55.363454: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-30 21:49:55.363777: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-30 21:49:55.365457: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/kf/9vc6x6n97q942grwf1zf0trw0000gn/T/tmpg96ejf4t
2023-11-30 21:49:55.368079: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-30 21:49:55.368101: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/kf/9vc6x6n97q942grwf1zf0trw0000gn/T/tmpg96ejf4t
2023-11-30 21:49:55.373998: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-30 21:49:55.376370: I tensorflow/cc/saved_model/load

8176

## Inference test

In [20]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [21]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [22]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [23]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 1.19 ms, sys: 2.14 ms, total: 3.32 ms
Wall time: 5.37 ms


In [24]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.7255490e-03 5.0721752e-08 4.1267772e-06 9.6219352e-05 1.4876920e-07
 9.9813098e-01 8.2815510e-17 2.6720117e-09 8.2250688e-13 4.1762651e-05
 3.0143952e-16 1.1715595e-06]
5


## Download model

In [25]:
!zip -r model.zip keypoint_classifier  

	zip warning: name not matched: keypoint_classifier

zip error: Nothing to do! (try: zip -r model.zip . -i keypoint_classifier)




---


# ❗️Hyperparameters Tuning

In [26]:
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

In [27]:
# Init parameters to tune
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([16, 32, 64]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([8, 16, 32]))
HP_NUM_UNITS_3 = hp.HParam('num_units_3', hp.Discrete([8, 16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

In [28]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_1,HP_NUM_UNITS_2,HP_NUM_UNITS_3, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [29]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False, save_best_only=True)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=15, verbose=1)

In [30]:
def train_test_model(hparams):

  model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_1], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_2], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS_3], activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
  
  model.compile(
    optimizer=hparams[HP_OPTIMIZER],
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
  
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False, save_best_only=True)

  model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[
               cp_callback,
               es_callback,
               ]
  ) 

  # Load model with best accuracy
  model = tf.keras.models.load_model(model_save_path)

  _, accuracy = model.evaluate(X_test, y_test)
  return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [31]:
session_num = 0

for num_units_1 in HP_NUM_UNITS_1.domain.values:
  for num_units_2 in HP_NUM_UNITS_2.domain.values:
    for num_units_3 in HP_NUM_UNITS_3.domain.values:
      for dropout_rate in np.arange(HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value, 0.1):
        for optimizer in HP_OPTIMIZER.domain.values:
          hparams = {
              HP_NUM_UNITS_1: num_units_1,
              HP_NUM_UNITS_2: num_units_2,
              HP_NUM_UNITS_3: num_units_3,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1

--- Starting trial: run-0
{'num_units_1': 16, 'num_units_2': 8, 'num_units_3': 8, 'dropout': 0.0, 'optimizer': 'adam'}
Epoch 1/50
82/88 [==========================>...] - ETA: 0s - loss: 2.3914 - accuracy: 0.2003
Epoch 1: val_loss improved from inf to 2.16656, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 2s 6ms/step - loss: 2.3804 - accuracy: 0.2038 - val_loss: 2.1666 - val_accuracy: 0.2591
Epoch 2/50
81/88 [==========================>...] - ETA: 0s - loss: 2.0078 - accuracy: 0.2859

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 2.16656 to 1.80112, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 3ms/step - loss: 1.9927 - accuracy: 0.2898 - val_loss: 1.8011 - val_accuracy: 0.3306
Epoch 3/50
76/88 [========================>.....] - ETA: 0s - loss: 1.7046 - accuracy: 0.3694
Epoch 3: val_loss improved from 1.80112 to 1.58134, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 4ms/step - loss: 1.6972 - accuracy: 0.3735 - val_loss: 1.5813 - val_accuracy: 0.4258
Epoch 4/50
83/88 [===========================>..] - ETA: 0s - loss: 1.5029 - accuracy: 0.4162
Epoch 4: val_loss improved from 1.58134 to 1.38424, saving model to keypoint_classifier.hdf5
88/88 [==============================] - 0s 3ms/step - loss: 1.4927 - accuracy: 0.4203 - val_loss: 1.3842 - val_accuracy: 0.4796
Epoch 5/50
83/88 [===========================>..] - ETA: 0s - loss: 1.3040 - accuracy: 0.5296
Epoch 5: val_loss improved from 1.38424 to 1.19919,

KeyboardInterrupt: 

In [ ]:
# !ATTENTION! Works only in Colab
%tensorboard --logdir logs/hparam_tuning

In [ ]:
!rm -rf logs